<a href="https://colab.research.google.com/github/DIPAKLIC/ImageAI/blob/main/Image_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **<center><u>Image AI</u></center>**
### *This is a program to predict the best image from a bunch of images.*

---



### *Tips to get started:-*

*1) Each time the runtime resets you need to run the first cell <b>" 1) Setup Environment "</b>*

*2) Upload the images by running the second cell<b>" 2) Upload Images "</b>*

*3) After the images are uploaded run the third cell<b>" 3) Predict "</b>*

*4) The best image predicted will be shown with its location.*

*5) If you want to run the program again run or clear the uploaded images the fourth cell <b>" 4) Clear Data Folder "</b> to clear the data files. (optional)*



## **1) Setup Environment**

In [ ]:
!pip install torch opencv-python
!git clone https://github.com/DIPAKLIC/ImageAI.git
import os
image_path = r'ImageAI/Images' 
if not os.path.exists(image_path):
    os.makedirs(image_path)

## **2) Upload Images**

In [ ]:
from google.colab import files
import shutil

uploaded = files.upload()
for file in uploaded:
  shutil.move(file, 'ImageAI/Images')

## **3) Predict**

In [ ]:
import argparse
import torch
import torchvision.models
import torchvision.transforms as transforms
import glob
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import Image as show_image
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

addr=[]
val=[]
def prepare_image(image):
    if image.mode != 'RGB':
        image = image.convert("RGB")
    Transform = transforms.Compose([
            transforms.Resize([224,224]),      
            transforms.ToTensor(),
            ])
    image = Transform(image)   
    image = image.unsqueeze(0)
    return image.to(device)

def predict(image, model, item):
    image = prepare_image(image)
    with torch.no_grad():
        preds = model(image)
    print(item + "'s " r'rating is: %.2f' % preds.item())
    localValue = '%.2f' % preds.item()
    addr.append(item)
    val.append(localValue)

files = glob.glob("ImageAI/Images/*.*")
print("Please wait for the program to predict, It might take a while depending on the device's performance")
for item in files: 
    parser = argparse.ArgumentParser()
    image = Image.open(item)
    model = torchvision.models.resnet50()
    model.fc = torch.nn.Linear(in_features=2048, out_features=1)
    model.load_state_dict(torch.load('ImageAI/model/model-resnet50.pth', map_location=device)) 
    model.eval().to(device)
    predict(image, model, item)
maxVal = max(val);
index = val.index(maxVal)
location = addr[index]
img = cv2.imread(location, cv2.IMREAD_UNCHANGED)
cv2_imshow(img)
print('The best image is '+'"'+ location + '"' + ' with a predicted rating of '+maxVal)

## **4) Clear Data Folder**

In [ ]:
#Function to clear the data folders
def clearImg():
    tmpFiles = glob.glob('ImageAI/Images/*.*')
    for file in tmpFiles:
        os.remove(file)

clearImg()